<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw5/nlp_hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Библиотеки и установки

In [ ]:
!pip3 install gensim pyLDAvis

In [ ]:
!pip install bigartm
!pip install corus
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_sm

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [ ]:
import spacy
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import numpy as np
import artm

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from corus import load_lenta
from gensim import corpora

In [ ]:
RANDOM_STATE = 111

# Загрузка данных

In [ ]:
records = load_lenta('lenta-ru-news.csv.gz')

In [ ]:
dataset = [sample.text.lower() for sample in records][:1000]

In [ ]:
dataset[0]

'вице-премьер по социальным вопросам татьяна голикова рассказала, в каких регионах россии зафиксирована наиболее высокая смертность от рака, сообщает риа новости. по словам голиковой, чаще всего онкологические заболевания становились причиной смерти в псковской, тверской, тульской и орловской областях, а также в севастополе. вице-премьер напомнила, что главные факторы смертности в россии — рак и болезни системы кровообращения. в начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. по данным росстата, в 2017 году от рака умерли 289 тысяч человек. это на 3,5 процента меньше, чем годом ранее.'

# Подготовка дадасета

In [ ]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
clean_process = lambda x: [
    token.lemma_.lower() for token in nlp(x) if
    not token.is_stop
    and not token.is_punct
    and not token.is_digit
    and not token.like_email
    and not token.like_num
    and not token.is_space
]

In [ ]:
clean_dataset = []

for text in tqdm(dataset):
    tokenize_text = clean_process(text)
    if tokenize_text:
        clean_dataset.append(clean_process(text))

100%|██████████| 1000/1000 [04:44<00:00,  3.51it/s]


In [ ]:
bigram = gensim.models.Phrases(clean_dataset, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[clean_dataset], threshold=100)  # Phraser == FrozenPhrases

In [ ]:
def get_ngrams(texts_out):
    texts_out = [trigram[bigram[doc]] for doc in texts_out]
    return texts_out

In [ ]:
data_ready = get_ngrams(clean_dataset)

In [ ]:
id2word = corpora.Dictionary(data_ready)

In [ ]:
# doc2bow() - вернет кортеж {id_слова: сколько_встречался}
corpus = [id2word.doc2bow(text) for text in data_ready]

# Обучение моделей

## LDA 1

Случайные параметры

In [76]:
%%time

lda_model_1 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=3,
    random_state=RANDOM_STATE,
    update_every=1, # Number of documents to be iterated through for each update
    chunksize=10, # == batch size
    passes=100, # Number of passes through the corpus during training
    alpha='symmetric', # A-priori belief on document-topic distribution
    iterations=150, # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus
    per_word_topics=True,
)

print(lda_model_1.print_topics())

[(0, '0.007*"год" + 0.005*"сообщать" + 0.004*"время" + 0.004*"женщина" + 0.004*"декабрь" + 0.003*"мужчина" + 0.003*"ребёнок" + 0.003*"полиция" + 0.003*"человек" + 0.003*"фотография"'), (1, '0.013*"россия" + 0.012*"год" + 0.010*"российский" + 0.006*"страна" + 0.005*"слово" + 0.005*"заявить" + 0.005*"декабрь" + 0.004*"время" + 0.004*"президент" + 0.004*"москва"'), (2, '0.013*"год" + 0.009*"компания" + 0.008*"рубль" + 0.006*"процент" + 0.004*"ракета" + 0.004*"цена" + 0.003*"составить" + 0.003*"исследование" + 0.003*"новый" + 0.003*"земля"')]
CPU times: user 2min 12s, sys: 1.61 s, total: 2min 14s
Wall time: 2min 21s


In [77]:
coher_args_1 = {
    'texts': clean_dataset,
    'dictionary': id2word,
    'model': lda_model_1
}
coherence_model_1 = gensim.models.coherencemodel.CoherenceModel(**coher_args_1)

In [78]:
coherence_model_1.get_coherence()

0.35772606148455105

### LDA 2

Отрегулируем количество тем. Для этого подумаем, какие темы в российских новостях встречаются чаще всего:
- военные конфликты
- политика
- преступления
- медицина
- наука

Предположим, что первые две темы встречаются чаще всего. Можно сделать предположение о распределении тем (параметр alpha) - [0.3, 0.4, 0.1, 0.5, 0.4]

In [82]:
%%time

lda_model_2 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    random_state=RANDOM_STATE,
    update_every=1, # Number of documents to be iterated through for each update
    chunksize=10, # == batch size
    passes=100, # Number of passes through the corpus during training
    alpha=[0.3, 0.4, 0.1, 0.5, 0.4], # A-priori belief on document-topic distribution
    iterations=150, # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus
    per_word_topics=True,
)

print(lda_model_2.print_topics())

[(0, '0.006*"декабрь" + 0.005*"загитов" + 0.005*"директор" + 0.005*"имя" + 0.005*"конкурс" + 0.004*"год" + 0.004*"актёр" + 0.004*"матч" + 0.004*"написать" + 0.004*"победить"'), (1, '0.007*"полиция" + 0.007*"ребёнок" + 0.007*"сообщать" + 0.007*"женщина" + 0.007*"время" + 0.006*"мужчина" + 0.006*"человек" + 0.005*"сообщаться" + 0.005*"the" + 0.004*"пользователь"'), (2, '0.019*"украина" + 0.017*"сотрудник" + 0.009*"фотография" + 0.006*"служба" + 0.006*"пост" + 0.006*"руководитель" + 0.006*"фсб" + 0.005*"проверка" + 0.005*"судно" + 0.005*"керченский_пролив"'), (3, '0.020*"год" + 0.009*"россия" + 0.007*"декабрь" + 0.007*"слово" + 0.007*"ноябрь" + 0.006*"москва" + 0.006*"заявить" + 0.005*"день" + 0.005*"сообщать" + 0.005*"рубль"'), (4, '0.015*"год" + 0.014*"россия" + 0.013*"российский" + 0.010*"страна" + 0.007*"президент" + 0.006*"компания" + 0.006*"процент" + 0.005*"время" + 0.005*"являться" + 0.005*"проект"')]
CPU times: user 3min, sys: 1.94 s, total: 3min 2s
Wall time: 3min 12s


In [83]:
coher_args_2 = {
    'texts': clean_dataset,
    'dictionary': id2word,
    'model': lda_model_2
}
coherence_model_2 = gensim.models.coherencemodel.CoherenceModel(**coher_args_2)

In [86]:
coherence_model_2.get_coherence()

0.31484551055232606


### LDA 3

In [87]:
%%time

lda_model_3 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=4,
    random_state=RANDOM_STATE,
    update_every=1, # Number of documents to be iterated through for each update
    chunksize=15, # == batch size
    passes=130, # Number of passes through the corpus during training
    alpha='symmetric', # A-priori belief on document-topic distribution
    iterations=200, # Maximum number of iterations through the corpus when inferring the topic distribution of a corpus
    per_word_topics=True,
)

print(lda_model_3.print_topics())

[(0, '0.008*"год" + 0.006*"декабрь" + 0.005*"сообщать" + 0.005*"время" + 0.005*"слово" + 0.004*"ребёнок" + 0.004*"женщина" + 0.004*"место" + 0.004*"человек" + 0.004*"мужчина"'), (1, '0.005*"год" + 0.005*"американский" + 0.005*"фотография" + 0.005*"пользователь" + 0.005*"ракета" + 0.004*"километр" + 0.004*"париж" + 0.003*"новый" + 0.003*"земля" + 0.003*"дальность"'), (2, '0.016*"год" + 0.016*"россия" + 0.010*"российский" + 0.008*"страна" + 0.006*"заявить" + 0.005*"компания" + 0.005*"слово" + 0.005*"президент" + 0.005*"декабрь" + 0.004*"украина"'), (3, '0.010*"сотрудник" + 0.009*"год" + 0.008*"суд" + 0.007*"полиция" + 0.007*"задержать" + 0.006*"информация" + 0.005*"дело" + 0.005*"человек" + 0.005*"сообщаться" + 0.004*"установить"')]
CPU times: user 2min 35s, sys: 1.87 s, total: 2min 37s
Wall time: 2min 57s


In [89]:
coher_args_3 = {
    'texts': clean_dataset,
    'dictionary': id2word,
    'model': lda_model_3,
}
coherence_model_3 = gensim.models.coherencemodel.CoherenceModel(**coher_args_3)

In [90]:
coherence_model_3.get_coherence()

0.33794751238187953

## Сравнение

- LDA_1: 0.357
- LDA_2: 0.314 (иногда nan)
- LDA_3: 0.338

## Визуализация через pyLDAvis

In [91]:
pyLDAvis.enable_notebook()

### LDA 1

In [92]:
vis_1 = gensimvis.prepare(lda_model_1, corpus, dictionary=lda_model_1.id2word)
vis_1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.193682 -0.046360       1        1  52.826684
0      0.137468 -0.142582       2        1  30.452722
2      0.056215  0.188942       3        1  16.720594, topic_info=            Term        Freq       Total Category  logprob  loglift
23        россия  734.000000  734.000000  Default  30.0000  30.0000
98    российский  563.000000  563.000000  Default  29.0000  29.0000
946     компания  244.000000  244.000000  Default  28.0000  28.0000
739        рубль  180.000000  180.000000  Default  27.0000  27.0000
106       страна  374.000000  374.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
287        новый   59.182262  230.203801   Topic3  -5.7406   0.4302
1332   оказаться   44.875890  103.210097   Topic3  -6.0174   0.9557
168      эксперт   37.201728   53.322572   Topic3  -6.2049   1.4285
27       система   38.609457   66.497269   Topic3  -6.1678   1.2449
674     являться   37.562791  139.398979   Topic3  -6.1953   0.4772

[191 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
12156      3  0.981168     apple
17596      1  0.054005   voyager
17596      2  0.027003   voyager
17596      3  0.918092   voyager
2958       2  0.986857     актёр
...      ...       ...       ...
168        1  0.300061   эксперт
168        3  0.693890   эксперт
674        1  0.674323  являться
674        2  0.057389  являться
674        3  0.272599  являться

[230 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

### LDA 2

In [93]:
vis_2 = gensimvis.prepare(lda_model_2, corpus, dictionary=lda_model_2.id2word)
vis_2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.237227 -0.182121       1        1  35.962166
4      0.152322  0.238802       2        1  27.213229
1     -0.139368 -0.056876       3        1  17.126731
0     -0.113592 -0.046565       4        1  12.760488
2     -0.136590  0.046760       5        1   6.937387, topic_info=             Term         Freq        Total Category  logprob  loglift
23         россия   786.000000   786.000000  Default  30.0000  30.0000
425       украина   145.000000   145.000000  Default  29.0000  29.0000
6             год  1290.000000  1290.000000  Default  28.0000  28.0000
154     сотрудник   128.000000   128.000000  Default  27.0000  27.0000
98     российский   593.000000   593.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
9145         порт    24.117872    25.012765   Topic5  -5.7586   2.6318
6689        судно    39.978884    50.020850   Topic5  -5.2532   2.4442
12121    вольский    25.311469    26.898424   Topic5  -5.7103   2.6074
4778          фсб    43.666076    73.513490   Topic5  -5.1650   2.1473
4754      корабль    39.489070    66.375039   Topic5  -5.2655   2.1489

[243 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1370      5  0.990539    facebook
2039      3  0.993838         the
1451      5  0.960462  автомобиль
3333      5  0.988857     аккаунт
2958      4  0.984443       актёр
...     ...       ...         ...
1103      4  0.981144   чемпионат
1751      3  0.985163        штат
3060      2  0.988575   экономика
168       2  0.981406     эксперт
674       2  0.993951    являться

[262 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 1, 3])

### LDA 3

In [94]:
vis_3 = gensimvis.prepare(lda_model_3, corpus, dictionary=lda_model_3.id2word)
vis_3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.220887 -0.131757       1        1  43.064175
0      0.069623  0.228060       2        1  28.748522
3     -0.130728 -0.074436       3        1  14.567963
1     -0.159782 -0.021868       4        1  13.619341, topic_info=              Term        Freq       Total Category  logprob  loglift
23          россия  773.000000  773.000000  Default  30.0000  30.0000
106         страна  383.000000  383.000000  Default  29.0000  29.0000
154      сотрудник  156.000000  156.000000  Default  28.0000  28.0000
919            суд  130.000000  130.000000  Default  27.0000  27.0000
98      российский  552.000000  552.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
137   опубликовать   38.811886  134.541002   Topic4  -5.9574   0.7505
886        издание   39.299467  168.540537   Topic4  -5.9449   0.5377
1943           вид   32.643087   81.528761   Topic4  -6.1305   1.0784
345          часть   32.132294   85.416697   Topic4  -6.1463   1.0160
179          время   33.594937  416.007370   Topic4  -6.1017  -0.5226

[245 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
2039       2  0.538448         the
2039       4  0.452786         the
17596      1  0.031591     voyager
17596      4  0.979336     voyager
1451       3  0.974512  автомобиль
...      ...       ...         ...
346        1  0.903171       число
346        2  0.094244       число
2554       4  0.973244         шоу
4276       4  0.970375       экран
3323       4  0.943001        юзер

[286 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 4, 2])

Из-за близких тем у второй и третьей модели коэффициент согласованности у них вышел меньше

# BigARTM

## Подготовка данных

In [95]:
tokenized = [' '.join(text) for text in clean_dataset]

In [96]:
cv = CountVectorizer(max_features=1000, max_df=0.8, min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform(tokenized).todense()).T

In [97]:
token_list = [i for i in cv.vocabulary_.keys()]

In [ ]:
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=token_list)

## Подбор гиперпараметров

In [99]:
def objective(space):
    model = artm.ARTM(num_topics=space['num_topics'], dictionary=bv.dictionary, cache_theta=True)

    model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))
    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(
          name='SparsePhi',
          tau=space['phi_tau']
      ),
    )
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(
            name='SparseTheta',
            tau=space['theta_tau']
        ),
    )
    model.regularizers.add(
        artm.DecorrelatorPhiRegularizer(
          name='DecorrelatorPhi',
          tau=space['decorrelation_tau']
        ),
    )

    model.fit_offline(bv, num_collection_passes=30)

    perplexity_score = model.score_tracker["perplexity_score"].last_value
    print("Perplexity:", perplexity_score)
    return {'loss': perplexity_score, 'status': STATUS_OK }

In [100]:
topics_to_check =  [5, 10, 15, 20]

In [101]:
def run_hyperparams_search():
    space={
        'num_topics': hp.choice('num_topics', topics_to_check),
        'phi_tau': hp.uniform('SparsePhi', -1, 1),
        'theta_tau': hp.uniform('SparseTheta', -1, 1),
        'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
    }

    trials = Trials()

    best_hyperparams = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=20,
        trials=trials,
    )
    return best_hyperparams

In [102]:
%%time
best = run_hyperparams_search()

Perplexity:
432.21343994140625
Perplexity:
503.0457763671875
Perplexity:
542.5831909179688
Perplexity:
498.405029296875
Perplexity:
337.319091796875
Perplexity:
491.1708984375
Perplexity:
342.3922424316406
Perplexity:
497.4382019042969
Perplexity:
409.12567138671875
Perplexity:
404.22393798828125
Perplexity:
507.2091979980469
Perplexity:
414.714111328125
Perplexity:
372.2765197753906
Perplexity:
390.1556091308594
Perplexity:
519.9230346679688
Perplexity:
432.693359375
Perplexity:
353.7129211425781
Perplexity:
404.26824951171875
Perplexity:
550.794677734375
Perplexity:
434.5757751464844
100%|██████████| 20/20 [00:44<00:00,  2.22s/trial, best loss: 337.319091796875]
CPU times: user 37.5 s, sys: 918 ms, total: 38.5 s
Wall time: 44.5 s


In [103]:
best

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'DecorrelatorPhi': 26233.038309047555,
 'SparsePhi': 0.5717942310781645,
 'SparseTheta': -0.7369635345121623,
 'num_topics': 3}

## Обучение модели

In [104]:
def fit_model(seed=RANDOM_STATE):
    model = artm.ARTM(num_topics=topics_to_check[best['num_topics']], dictionary=bv.dictionary, cache_theta=True, seed=seed)
    model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                          dictionary=bv.dictionary))

    model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
    model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
    model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

    model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=best['SparsePhi'])) # сглаживание/разреживание матрицы Phi
    model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=best['SparseTheta'])) # сглаживание/разреживание матрицы Theta
    model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=best['DecorrelatorPhi'])) # сделать темы более разнообразными

    model.fit_offline(bv, num_collection_passes=30)
    return model

In [105]:
%%time

model = fit_model()
model.score_tracker["perplexity_score"].last_value

CPU times: user 1.71 s, sys: 35.2 ms, total: 1.75 s
Wall time: 1.75 s


335.158935546875

## Визуализация

In [ ]:
def prepare_vis_data():
    phi = model.get_phi()
    theta = model.get_theta().to_numpy().T
    theta = theta / theta.sum(axis=1, keepdims=1)
    data = {'topic_term_dists': phi.to_numpy().T,
            'doc_topic_dists': theta,
            'doc_lengths': n_wd.sum(axis=0).tolist(),
            'vocab': phi.T.columns,
            'term_frequency': n_wd.sum(axis=1).tolist()}
    return data

In [ ]:
model_data = prepare_vis_data()
model_vis = pyLDAvis.prepare(**model_data)

In [108]:
pyLDAvis.display(model_vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Вывод

У BigARTM более быстрое обучение, чем у LDA.

Темы определены сравнительно хорошо.